# Code zu Lanczos-Verfahren mithilfe von QR-Verfahren

In [10]:
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import block_diag
from scipy.linalg import qr
import time
import matplotlib.pyplot as plt

### Funktionen zum generiern von Matrizen

In [11]:
def gen_hess(n, compl = False):
    if compl:
        a = np.random.rand(n) + np.random.rand(n)*1j
        b = np.random.rand(n-1) + np.random.rand(n-1)*1j
        c = np.random.rand(n-1) + np.random.rand(n-1)*1j
        return np.diag(b, -1) + np.diag(a,0) + np.diag(c, 1)
    else:
        A = np.zeros((n,n))
        a = np.random.rand(n)*50
        A = A + np.diag(a)
        for j in range(n-1):
            b = np.random.rand(n-j-1)*50
            A = A + np.diag(b,j+1)
        return np.diag(np.random.rand(n-1)*50,-1) +  A
    

def gen_upper_tri(eig, compl = False):
    n = eig.shape[0]
    if compl:
        return np.triu(np.random.rand(n,n) + np.random.rand(n,n) * 1j, 1) + np.diag(eig)
    else:
        return np.triu(np.random.rand(n,n),1) + np.diag(eig)
    

def gen_hermite(eig, compl = False):
    n = eig.shape[0]
    B = unitary_group.rvs(n)
    return B.T.conj()@np.diag(eig)@B


def gen_rand_mat(eig, compl = False):
    n = eig.shape[0]
    if compl:
        B = np.random.rand(n,n) + np.random.rand(n,n)*compl*1j
    else:
        B = np.random.rand(n,n)
    return np.linalg.inv(B)@np.diag(eig)@B



#print(gen_hess(5))
#print(gen_upper_tri(np.array([1,2,3]), True))
#print(gen_hermite(np.array([1,2,3,4])))
#print(gen_rand_mat(np.array([1,2,7,4])))



### Verfahren implementiert

In [12]:
def QR_simple(A,tol = 1e-6):
    count = 0
    while abs(A[1,0]) > tol:
        Q,R = np.linalg.qr(A)
        A = R@Q
        count +=1
    return A, sorted(np.diag(A)) ,count



def QR_shift(A,tol=1e-10):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol:
            rho = A[i,i]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count +=1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_shift2(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_hesse_old(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = QR_decomp_hesse_old(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count

def QR_hesse(A,tol=1e-14):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            A = Hessenberg_QR_RQ(A-rho*np.identity(n))            #Statt vorimplementierter QR-Zerlegung hier QR-Zerlegung mit Givens-Rotationen
            A += rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count

def QR_decomp_hesse(A):
    n = A.shape[0]
    Q = np.eye(n,n, dtype = complex)
    
    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
      
        for j in range(n):
            if j < i:
                temp_2 = Q[i,j]
                Q[i,j] = c.conj()*temp_2 + s.conj()*Q[i+1,j]
                Q[i+1,j] = -s*temp_2 + c*Q[i+1,j]
            else:
                temp_1 = A[i,j]
                A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
                A[i+1,j] = -s*temp_1 + c*A[i+1,j]
                temp_2 = Q[i,j]
                Q[i,j] = c.conj()*temp_2 + s.conj()*Q[i+1,j]
                Q[i+1,j] = -s*temp_2 + c*Q[i+1,j]
    
    return Q.T.conj(), A

def QR_decomp_hesse_old(A):
    n = A.shape[0]
    Q = np.eye(n,n)

    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
            
        M = np.array([[c.conj(), s.conj()], [-s, c]])
        G = block_diag(np.eye(i,i), M, np.eye(n-i-2, n-i-2))
        Q = G@Q
        for j in range(i, n):
            temp_1 = A[i,j]
            A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
            A[i+1,j] = -s*temp_1 + c*A[i+1,j]
    return Q.T.conj(), A


def Hessenberg_QR_RQ(A):
    n = A.shape[0]
    c_arr = np.zeros(n, dtype = complex)
    s_arr = np.zeros(n, dtype = complex)
    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):                     
            t = A[i+1,i]/abs(A[i,i])                         
            root = (1+abs(t)**2)**(1/2)                     
            c_arr[i] = A[i,i]/(abs(A[i,i])*root)                  
            s_arr[i] = t/root          
        else:
            t = A[i,i]/abs(A[i+1,i])                 
            root = (1+abs(t)**2)**(1/2)                      
            s_arr[i] = A[i+1,i]/(abs(A[i+1,i])*root)               
            c_arr[i] = t/root                              
            
        for j in range(i,n):
                temp_1 = A[i,j]
                A[i,j] = c_arr[i].conj()*temp_1 + s_arr[i].conj()*A[i+1,j] 
                A[i+1,j] = -s_arr[i]*temp_1 + c_arr[i]*A[i+1,j]
                
    for i in range(0,n-1):
        for j in range(0,i+2):
            temp_1 = A[j,i]
            A[j,i] = c_arr[i]*temp_1 + s_arr[i]*A[j,i+1]
            A[j,i+1] = -s_arr[i].conj()*temp_1 + c_arr[i].conj()*A[j,i+1]
    
    return A


def lanczos(A,k = 0):
    n = A.shape[1]
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    gam = [v[0].T.conj()@A@v[0]]
    w = (A - gam[0]*np.identity(n))@v[0]
    delta = [np.linalg.norm(w)]
    i = 0
    while delta[i] > 1e-10 and i < k-1:
        v.append(w/delta[i])
        i +=1
        gam.append(v[i].T.conj()@A@v[i])
        w = (A - gam[i]*np.identity(n))@v[i] - delta[i-1]*v[i-1]
        delta.append(np.linalg.norm(w))
    T = np.diag(delta[:-1], -1) + np.diag(gam) + np.diag(delta[:-1], 1)
    return QR_hesse(T)

def arnoldi(A, dim, k = 0):
    n = dim
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    h = np.zeros((k,k))
    
    for j in range(k):
        w = A(v[j])
        for l in range(j+1):
            h[l][j] = w.T.conj()@v[l]
            w = w - h[l][j]*v[l]
        if j < k-1:
            h[j+1][j] = np.sqrt(w.T.conj()@w)
            if abs(h[j+1][j]) < 1e-2:
                return QR_hesse(h[:j+1,:j+1])
            else:
                v.append(w/h[j+1][j])
    #print(sorted(np.linalg.eigvals(h)))
    return QR_hesse(h)

### Testing

In [18]:
def error(eig_1, eig_2):
    eig_1 = np.sort(eig_1)
    eig_2 = np.sort(eig_2)
    return np.linalg.norm(eig_1 - eig_2)


def runtime(f,A):
    start = time.time()
    f(A)
    end = time.time()
    return end - start

def runtime_error(f,A):
    start = time.time()
    S = f(A)
    end = time.time()
    return end - start, S

def runtime_error_lanczos(f,A,k):
    start = time.time()
    S = f(A,k)
    end = time.time()
    return end - start, S

def testing(f, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error(f,A)
        err = error(A[1], eigv)
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()

    
def testing_lanczos(f,k, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error_lanczos(f,A,k)
        err = error(A[1][0], eigv[0])
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime lanczos")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()
    
def selective_error(eig_1, eig_2, k = 1):
    n_1 = len(eig_1)
    n_2 = len(eig_2)
    error_1 = np.linalg.norm(eig_1[:k] - eig_2[:k])
    error_2 = np.linalg.norm(eig_1[:n_1-k-1:-1] - eig_2[:n_2-k-1:-1])
    return error_1 + error_2

# ------- testing starts here -------

n = 1000

mu, sigma = 3, 1
eigv = np.sort(np.random.rand(n)*400)

#eigv = np.array([-(i+1)*10000 for i in range(n)])
A = gen_hermite(eigv)

def matmul(v):
    return A@v


print(np.sort(eigv))
#print(np.round(QR_simple(A)[1],3))
#print(np.round(QR_shift(A)[1],3))
#print(np.round(QR_shift2(A)[1],3))
#print(np.round(QR_hesse(A)[0],3))
#print(np.round(lanczos(A,30)[1],3))
#print(np.linalg.eigvals(A))
#print(np.sort(np.round(arnoldi(matmul,n)[1], 3)))

#print(error(QR_simple(A)[1], eigv))
#print(error(QR_shift(A)[1], eigv))
#print(error(QR_shift2(A)[1], eigv))
#print(error(lanczos(A)[1], eigv))

#print(error(arnoldi(lambda x: A@x,n, 50)[1],eigv))
#print(arnoldi(lambda x: A@x,n, 50)[1])
#print(runtime(QR_simple, A))
#print(runtime(QR_shift, A))
#print(runtime(QR_shift2, A))
#print(runtime_error_lanczos(lanczos, A, 10)[0])


[3.02898927e-01 5.53032566e-01 8.44911015e-01 1.04834382e+00
 2.34605487e+00 2.98275097e+00 3.01439159e+00 4.78309649e+00
 5.02350015e+00 5.81764568e+00 6.39170470e+00 6.50759341e+00
 6.76441839e+00 6.94347206e+00 7.55347516e+00 9.57674378e+00
 1.09491083e+01 1.11379985e+01 1.15665133e+01 1.21554352e+01
 1.22044946e+01 1.26926015e+01 1.27054918e+01 1.28691268e+01
 1.37668966e+01 1.37863860e+01 1.38945216e+01 1.48780083e+01
 1.55149631e+01 1.60721045e+01 1.62885008e+01 1.69003020e+01
 1.76497211e+01 1.79492304e+01 1.82691133e+01 1.84461764e+01
 1.85906700e+01 1.90082685e+01 1.93047643e+01 1.93915408e+01
 1.99483201e+01 2.01096165e+01 2.02615215e+01 2.05868902e+01
 2.10444778e+01 2.11939324e+01 2.19550602e+01 2.20661697e+01
 2.21718121e+01 2.23657760e+01 2.30254596e+01 2.33603182e+01
 2.34262805e+01 2.41373397e+01 2.50396162e+01 2.55708381e+01
 2.59857344e+01 2.62118118e+01 2.67341113e+01 2.68873757e+01
 2.70683880e+01 2.73901468e+01 2.73982082e+01 2.87937367e+01
 2.88369171e+01 2.918674

In [19]:
k = 120
for k in range(5,50):
    lanczos_test = lanczos(A,k)[1]
    print(np.sort(lanczos_test))
    err_ = abs(lanczos_test[-1]-eigv[-1])


    print("lanczos = ", err_)

    arnoldi_test = arnoldi(lambda x: A@x,n,k)[1]
    #print(np.sort(arnoldi_test))
    error_1 = abs(arnoldi_test[-1]-eigv[-1])

    print("arnoldi=", error_1)

[ 21.34672395+1.17330830e-15j  93.60590027+2.37913731e-15j
 199.04118087+3.11001235e-15j 308.60239077+2.23447615e-15j
 380.84216584+1.20609546e-15j]
lanczos =  17.919400770217464
arnoldi= 20.35521037636022
[ 13.8122826 +3.63373699e-15j  66.90278885+1.85869371e-15j
 150.40191366+1.76246601e-15j 247.30051663+1.73099186e-15j
 333.18832625+1.88020942e-15j 384.52754904+3.56680133e-15j]
lanczos =  14.234017575858957
arnoldi= 13.840827811369934


c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:194: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:197: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:150: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:151: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:157: ComplexWarning: Casting complex

[ 10.95432155-1.29501748e-15j  51.61955613-1.72665425e-15j
 117.49353689-3.37791238e-16j 197.90837786-6.07294706e-16j
 280.56381321-3.16288807e-16j 348.21489833-1.74160925e-15j
 388.05597624-1.30281238e-15j]
lanczos =  10.705590378028546
arnoldi= 10.096129453905121
[  8.10999568+1.76404172e-15j  41.10114888+2.76039308e-15j
  96.12409667+2.49101289e-15j 162.84750904+6.09539253e-16j
 232.86229987+5.42622137e-16j 303.77527987+2.72251474e-15j
 358.26593769+2.67878443e-15j 389.6523734 +1.97421217e-15j]
lanczos =  9.109193213292087
arnoldi= 7.135866800118151
[  6.67415417+1.81568111e-15j  33.52587428+1.13544673e-15j
  76.60622705+1.49654281e-15j 133.32007856+1.72759392e-15j
 197.76677864+1.32120276e-15j 261.83354005+1.74340312e-15j
 324.35341058+1.70110631e-15j 365.38522959+1.12910831e-15j
 392.81358758+1.91872803e-15j]
lanczos =  5.947979032015269
arnoldi= 7.429522009798404
[  3.15747922+6.83519931e-16j  26.65777093-5.41167943e-16j
  63.00472662-4.74703696e-16j 114.10357642+3.13991117e-16j


arnoldi= 0.807548396816685
[  0.61514649+1.93758630e-16j   5.92106269-6.98055395e-16j
  13.69599128-6.51915207e-16j  23.90604924-5.15187946e-16j
  36.92955193+2.68303761e-16j  53.31516506+3.54495494e-16j
  69.20073337-3.19422395e-18j  94.01638375-2.15838841e-16j
 112.92171335+6.93959790e-16j 137.93707523+1.04547454e-16j
 162.24464044-7.86368444e-16j 187.39047686+1.13617377e-16j
 212.43867867+9.21176952e-17j 237.96973761-7.80811749e-16j
 262.20882339+5.12741371e-17j 286.73182342+7.31738320e-16j
 308.95707383-2.60957189e-16j 330.36863002-2.26655081e-18j
 347.4633342 +4.99502215e-16j 362.99194912+2.73771322e-16j
 375.14992174-2.92203176e-16j 387.33175477-6.84068040e-16j
 392.82175942-7.61105475e-16j 398.54120926+2.76710950e-16j]
lanczos =  0.22035735735056505
arnoldi= 0.46099958328676394
[  0.5405881 -5.70944668e-16j   5.35872201-7.89713743e-16j
  13.18243652-3.31412115e-16j  22.21288456-3.14411846e-16j
  34.66406197-2.30850351e-16j  49.50232996-1.16060446e-16j
  65.78198841+1.28130682e-1

arnoldi= 0.20703879281825266
[  0.43060334+5.50939683e-16j   2.73949159+1.38761151e-15j
   6.47871721+3.08171782e-16j  12.86212424+6.88604567e-16j
  19.65130434+1.06984942e-15j  27.71668482+8.78054067e-16j
  37.61851141+5.24278898e-16j  49.08017857+6.04509885e-16j
  60.73070574+3.29989789e-16j  72.24044457-1.41854811e-16j
  90.35014643+8.36529229e-16j 105.51474536+8.07906239e-16j
 119.37834636+3.63817029e-16j 136.63551887+1.07976980e-15j
 153.05752473+1.11365780e-15j 173.47210209+7.73938034e-16j
 189.85082828+7.79447520e-16j 208.36781965+7.18594582e-16j
 226.37047106+8.76566478e-16j 245.06782787+9.33128109e-16j
 262.77539119+7.82256450e-16j 279.12603025+6.28822775e-16j
 295.39827185+8.06011175e-16j 310.85895927+6.63486810e-16j
 324.52096024-2.14643616e-16j 338.56028221+5.27221409e-16j
 351.31158514+8.81425698e-16j 362.27193284+2.87896242e-16j
 371.85605509+1.05934802e-15j 379.70861151+1.07367313e-15j
 387.61031604+7.93545114e-16j 391.57291235-4.95378085e-18j
 396.66893862+1.60996850e-1

arnoldi= 0.012148184100681192
[  0.50972194+1.38201550e-15j   1.49464706+1.06536907e-15j
   3.91921454+1.56471204e-15j   6.95364984+4.40341868e-16j
  12.84934101+1.16713679e-15j  18.47802139+9.78050136e-16j
  24.49071999+7.25758090e-16j  31.85697233+1.51874345e-15j
  40.89393133+7.32480144e-16j  50.74243429+8.82968252e-16j
  59.7496508 +1.89084354e-15j  70.02353155+8.14880356e-16j
  81.52570363+6.13652433e-16j  95.07957799+1.71693741e-15j
 107.63596501+8.22253036e-16j 118.77479881+1.42680396e-15j
 134.1011712 +1.03891284e-15j 148.77942265+1.02918636e-15j
 163.53217648+9.53290516e-16j 177.57352253+7.41733873e-16j
 191.46543413+1.24031356e-15j 205.55890912+1.30159851e-15j
 222.02869176+5.75108039e-16j 234.767032  +1.03444686e-15j
 250.38544709+9.39099802e-16j 263.69597732+1.14905338e-15j
 278.74650716+1.41407326e-15j 291.63882427+6.94431953e-16j
 304.44107859+1.69032522e-15j 316.56249188+8.43524998e-16j
 329.5734141 +8.75489057e-16j 338.83927742+1.53400217e-15j
 349.11313792+1.03165041e-

arnoldi= 0.017056778716153076
[3.75688063e-01-3.75403488e-17j 9.44123635e-01-4.49144746e-16j
 3.01764960e+00-1.45546188e-16j 6.55915831e+00+3.51884002e-16j
 1.07970754e+01-9.31275816e-16j 1.40043577e+01-9.70517915e-16j
 1.94781696e+01+9.97509495e-17j 2.52974786e+01-4.91063663e-16j
 3.19213060e+01+2.60719239e-16j 3.92254318e+01-8.32926160e-16j
 4.70148507e+01+2.24736062e-16j 5.60616511e+01+1.61631286e-16j
 6.48847490e+01+1.22202191e-16j 7.40548762e+01-7.01871514e-17j
 8.49419194e+01+2.84055312e-16j 9.62457444e+01-7.32891810e-16j
 1.07494548e+02-3.41743314e-17j 1.17581468e+02+6.36688257e-16j
 1.30758736e+02-4.82381395e-16j 1.43472491e+02-5.21328390e-16j
 1.53973498e+02-1.60432989e-15j 1.68748185e+02-6.96323382e-16j
 1.79892595e+02-4.18301652e-16j 1.92219505e+02-1.16882409e-15j
 2.05075967e+02-8.75755344e-16j 2.19402853e+02-3.39779157e-16j
 2.31129155e+02-6.08311468e-16j 2.44908018e+02-1.52779707e-15j
 2.56924402e+02-2.76418548e-16j 2.68772555e+02-3.14121678e-16j
 2.81230844e+02+2.7514219

In [ ]:
k = [i*10 for i in range(1,60)]

n = len(k)
mat_array = [0]*n
mat_array_2 = [0]*n
for j in range(n):
    mat_array[j] = (gen_hess(k[j]))
    mat_array_2[j] = gen_rand_mat(np.random.rand(k[j])*500)
time_good = [0]*n
time_bad = [0]*n


In [ ]:
for j in range(len(k)):
    time_good[j] = runtime(QR_decomp_hesse, mat_array[j])
    time_bad[j] = runtime(np.linalg.qr, mat_array_2[j])

In [ ]:
%matplotlib notebook

plt.plot(k, time_good, '-o', label = "good")
plt.plot(k, time_bad, '-o',label = "bad")

plt.legend()
plt.show()

In [ ]:
n_dim_2 = 500 

eigvalues_2 = np.sort(2*np.random.rand(n_dim))
eigvalues_2[-1] = 3
eigvalues_2[-2] = 2.5
eigvalues_2[0] = -1
eigvalues_2[1] = -0.5
L = gen_hermite(eigvalues_2)

j = 10

In [ ]:
lanczos_arr = [[0]]*j

for l in range(j):
    lanczos_arr[l] = lanczos(L,l+2)[1]

In [ ]:
%matplotlib notebook

for l in range(n_dim):
    plt.plot(12,eigvalues[l], 'x')
for l in range(len(lanczos_arr)):
    for k in range(len(lanczos_arr[l])):
        plt.plot(l+2, lanczos_arr[l][k], 'x')
        
plt.axvline(x=11.5)
plt.xlabel("Kyrolvraumdimension, bei 12 Eigenwerte")
plt.ylabel("Wertebereich der Eigenwerte")
plt.show()